In [1]:
import re #импортируем библиотеку regex

In [ ]:
def get_date(text):  #функция принимает на вход текст судебного решения, возвращает дату его вынесения
    text = text.lower().replace('у с т а н о в и л', "установил").split('установил')[0].split('подсудим')[0]
    try:
        date = re.search(r'(?:\d+) (?:января|февраля|марта|апреля|мая|июня|июля|августа|сентября|октября|ноября|декабря) (?:19|20)\d\d', text).group(0)
        date = date.replace('января', '01').replace('февраля', '02').replace('марта', '03').replace('апреля', '04').replace('мая', '05').replace('июня', '06').replace('июля', '07').replace('августа', '08').replace('сентября', '09').replace('октября', '10').replace('ноября', '11').replace('декабря', '12').replace(' ', '.')
    except:
        try:
            date = re.search(r'([0-2]\d|3[01])\.(0\d|1[012])\.(\d{4})', text).group(0)
        except:
            date = None
    return date


def children(text):  #функция принимает на вход текст судебного решения, возвращает "да", если наличие ребенка/детей стало смягчающим обстоятельством для подсудимого, "нет", если нет
    s = text.split('\n')
    answ = ['да' for x in s if re.search(r'смягчающ\w+', x.lower()) and re.search(r'наличие', x.lower()) and (re.search(r'несовершеннолетн\w+', x.lower()) or re.search(r'ребенка', x.lower()) or re.search(r'детей', x.lower()))]
    if 'да' in answ:
        return 'да'
    else:
        return 'нет'
    
def prosba(text): #функция принимает на вход текст судебного решения, возвращает "да", если потерпевший просит суд не наказыват строго подсудимого
    
    s = text.split('\n')
    answ = ['да' for x in s if re.search(r'потерпевш\w+|ходатайство|просьб\w+|просит', x.lower()) and re.search(r'не\sнастаив\w+|не\sнаказ\w+|не\sпривлека\w+|не\sлишать|смягчи\w+', x.lower()) and re.search(r'строго|ответств\w+|подсудим\w+', x.lower()) or re.search(r'снисхожд\w+', x.lower())]
    if 'да' in answ:
        return 'да'
    else:
        return 'нет'
    
def previous_conviction(x):#функция принимает на вход текст судебного решения, возвращает "да", если подсудимый был ранее судим
    p_c = re.search(r'не судима*|судимост\w+ не име\w+|не имеющ\w+(?:\sнепогаш\w+)* судимост\w+', x)
    if p_c:
        return 'нет'
    else:
        if not re.search(r'ранее судима*|судимост\w+ име\w+|име\w+(?:\sнепогаш\w+)* судимост\w+|,*\sсудима*(?:ого|ой)*(?:\.|,|\s|\:)*|име\w+ судимост\w+', x):
            return 'нет'
    return 'да'

def confession(x): #функция принимает на вход текст судебного решения, возвращает "да", если подсудимый признал вину, "частично", если признал частично, и "нет, если не признал
    vina = re.compile(r".{100}(?:(?:свою|себя)*\s*вин\w+\s*(?:свою|себя)*\s*(?:(?:в (?:инкриминируемом\s*(?:ему|ей)*|указанном) (?:преступ\w+|деян\w+|преступ\w+\sдеян\w+))|(?:(?:по|в) предъявл\w+ обвинен\w+)|(?:в совершен\w+(?:\s(?:(?:\s*выше\s*)*(?:\s*указанного|изложенного)(?:\sвыше)*)|(?:\s*данного)|(?:\s*вышеопис\w+)|(?:\s*вышеназв\w+)|(?:\s*инкриминир\w+|\s*вменяемого)\s*(?:ему|ей)*)*\s*(?:преступ\w+|деян\w+|преступ\w+\sдеян\w+)(,*\s*предусмотренного\s(?:(?:ч\.|частью)\s*1\s*)*(?:ст\.|стать\w+)\s*117\s*(?:(?:ч\.|частью)\s*2\s*)*(?:ук|уголовного\s*кодекса)*\s*(?:рф|российской\s*федерации),*)*\s*(?:(?:при (?:указанных|изложенных|установленных) (выше|судом) обстоятельствах)|(?:,* изложенного в обвинительном заключении,*))*)|(?:\s*в (?:инкриминируемом деянии|содеянном)))*\s*(?:она*|подсуди\w+)*\s*(?:в судебном заседании)*(?:фактически\s)*(?:не)*(?:\s*полностью|\s+частичн\w+)*\s*призн\w+(?:\s*полн)*)|(?:(?:(?:призн\w+\s*полн\w+)|(?:(?:\s*полн\w+)*(?:\sне)*\s*призн\w+))\s*(?:(?:свою|себя)|(?:(?:им|ей|подсудим\w+)*(?:\sсвоей)*))*\sвин\w+)|(?:обвинение(?:,* с которым\s*(?:полностью)*соглас\w+\s(?:подсудим\w+)*)|(?: (?:ей|ему|подсудим\w+) понятно, (?:она*|подсудим\w+) (?:полностью)*\s*соглас\w+ с ним))|(?:(?:она*)*\s*(?:полностью)*\s*(?:не)*\s*согла\w+ с\s*(?:предъявленным)*\s*(?:ей|ему)* обвинением)|(?:(заявила* о своём согласии)*\s*с предъявленным обвинением(?:\sсоглас\w+\s*(?:полностью)*)*)|(?:не призн\w+ вин\w+)|(?:призна\w+ частичн\w+)|(?:частичн\w+ призн\w+)|(?:соглас\w+ в полном объеме).{100}")
    V = re.search(vina, x)
    if V:
        V = V.group()
        if re.search(r'не\sпризн\w+|не\sсогла\w+', V) and not re.search(r'(?:призн\w+ (?:полностью|(?:в полном объё*е*ме)))|(?:(?:полностью|(?:в полном объё*е*ме)) призн\w+)', V):
            return "нет", V
        else:
            if 'частичн' in V:
                return 'частично', V
            else:
                return "да", V
    else:
        return None, None        

def alco(x): #функция принимает на вход текст судебного решения, возвращает "да", если подсудимый был в состоянии алкогольного или наркотического опьянения
    al = r".{100}(?:в состоянии)*\s*(?:нарк\w+|алк\w+)\s*опьян\w+.{100}"
    A = re.search(al, x)
    if A:
        return 'да', A.group()
    return "нет", None